# __STRATEGIE D'ANALYSE BIOINFORMATIQUE - MENTORING PROJECT__

__AGMIMONHAN Attolou Raoul, NAME Pakyendou Estel__

__Tuteurs: Aurore COMTE & Sebastien RAVEL__

Jupyter inspired by the model created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)
***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>


[PRACTICE V - PCA - DAPC - OTHERS](#Genetic_dversity)

   
   

***

# __Practice V__

## Evaluate level of missing data (by sample, by positions)

In [ ]:
## Créer un répertoire PLINK dans le repertoire de travail

mkdir -p /scratch/MOryzae/PLINK

In [ ]:
## Se déplacer dans le répertoire créé

cd PLINK/

In [ ]:
## Charger le module plink avant tout

module load plink/1.9

In [ ]:
## Lancer l'évaluation des données manquantes

plink -vcf /scratch/MOryzae/SNP/vcf_files/snp_correct.vcf.gz --allow-extra-chr --missing --out  ./plink/dataset

## __Generate PCA using genotyping information contained in VCF__

In [ ]:
## Lancer le run

plink -vcf /scratch/MOryzae/SNP/vcf_files/snp_correct.vcf.gz --allow-extra-chr --cluster --matrix --pca 3 --mind --out ./plink/dataset

## __Convertissons le fichier "eigenvec" généré en format csv__

In [ ]:
## Se déplacer dans le répertoire créé

cd /scratch/MOryzae/SCRIPTS

In [ ]:
## Ouvrir l'éditeur de texte nano

nano eigenvec_to_csv.sh

In [ ]:
#!/bin/bash

############# SLURM Configuration ##############

### Define Job name
#SBATCH --job-name=eigenvec_to_csv

### Define partition to use
#SBATCH -p normal

### Define number of CPUs to use
#SBATCH -c 8

### Specify the node to run on
#SBATCH --nodelist=node20  # Run the job on node20

#################################################

########### Execution Command ###################

# Define directories
INPUT_DIR="/scratch/MOryzae/PLINK"
OUTPUT_DIR="/scratch/MOryzae/PLINK"

# List of directories to process
DIRECTORIES=("plink")

# Loop through each directory
for DIR in "${DIRECTORIES[@]}"; do
    PCA_FILE="$INPUT_DIR/$DIR/dataset.eigenvec"
    OUTPUT_CSV="$OUTPUT_DIR/$DIR/dataset.csv"

    # Check if PCA results exist
    if [ -f "$PCA_FILE" ]; then
        echo "Processing PCA results for $DIR..."

        # Convert eigenvec file to CSV format
        awk 'NR==1{print "FID,IID,PC1,PC2,PC3"} NR>1{print $1","$2","$3","$4","$5}' "$PCA_FILE" > "$OUTPUT_CSV"

        # Check if conversion was successful
        if [ $? -eq 0 ]; then
            echo "Conversion successful: $OUTPUT_CSV created."
        else
            echo "Error: Failed to convert $PCA_FILE to CSV."
            exit 1
        fi
    else
        echo "Error: PCA results file not found in $DIR."
        exit 1
    fi
done

echo "PCA analysis completed. Results are saved in $OUTPUT_DIR."


In [ ]:
## Lancer le script

sbash eigenvec_to_csv.sh

## __Sortir un plot pour la PCA afin de faciliter la visualisation__

In [ ]:
## Ouvrir l'éditeur de texte nano

nano pca_plot.sh

In [ ]:
#!/bin/bash

############# SLURM Configuration ##############

### Define Job name
#SBATCH --job-name=genome_pca_plot

### Define partition to use
#SBATCH -p normal

### Define number of CPUs to use
#SBATCH -c 8

### Specify the node to run on
#SBATCH --nodelist=node20

#################################################

########### Execution Command ###################

module load python/3.12.0  # Charge Python 3.12 sur le cluster

# Define directories
PCA_RESULTS_DIR="/scratch/MOryzae/PLINK"
OUTPUT_PLOT_DIR="/scratch/MOryzae/PLINK"

# List of directories to process
DIRECTORIES=("plink")

# Loop through each directory
for DIRECTORY in "${DIRECTORIES[@]}"; do
    PCA_FILE="$PCA_RESULTS_DIR/$DIRECTORY/dataset.eigenvec"
    OUTPUT_DIR="$OUTPUT_PLOT_DIR/$DIRECTORY"
    OUTPUT_PLOT_2D="$OUTPUT_DIR/dataset_2D.png"
    OUTPUT_PLOT_3D="$OUTPUT_DIR/dataset_3D.png"

    # Ensure the output directory exists
    mkdir -p "$OUTPUT_DIR"

    # Check if the PCA results file exists
    if [ -f "$PCA_FILE" ]; then
        echo "Processing PCA results for $DIRECTORY..."
        
        # Call the Python script to generate the plots
        python3 <<EOF
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os

# Define input and output paths
pca_results_file = "$PCA_FILE"
output_plot_2D = "$OUTPUT_PLOT_2D"
output_plot_3D = "$OUTPUT_PLOT_3D"

# Read PCA results
try:
    pca_results = pd.read_csv(pca_results_file, sep=r'\s+', header=None)
    pca_results.columns = ['FID', 'IID', 'PC1', 'PC2', 'PC3']
except Exception as e:
    print(f"Error reading PCA results file {pca_results_file}: {e}")
    exit(1)

# Plot 2D scatter plot for PC1 vs PC2
try:
    plt.figure(figsize=(8, 6))
    plt.scatter(pca_results['PC1'], pca_results['PC2'], s=100)
    plt.title('PCA Results: $DIRECTORY (2D)')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid()
    plt.savefig(output_plot_2D)
    plt.close()
except Exception as e:
    print(f"Error creating 2D plot for {pca_results_file}: {e}")
    exit(1)

# Plot 3D scatter plot for PC1, PC2, and PC3
try:
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(pca_results['PC1'], pca_results['PC2'], pca_results['PC3'], s=100, c='blue', alpha=0.7)
    ax.set_title('PCA Results: $DIRECTORY (3D)')
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    plt.savefig(output_plot_3D)
    plt.close()
except Exception as e:
    print(f"Error creating 3D plot for {pca_results_file}: {e}")
    exit(1)

# Verify plots were created
if not os.path.exists(output_plot_2D) or not os.path.exists(output_plot_3D):
    print(f"Error: Output plots not created for {pca_results_file}")
    exit(1)
EOF

        # Check if the Python script executed successfully
        if [ $? -eq 0 ]; then
            echo "Plots successfully created for $DIRECTORY: $OUTPUT_PLOT_2D, $OUTPUT_PLOT_3D"
        else
            echo "Error: Failed to create plots for $DIRECTORY."
            exit 1
        fi
    else
        echo "Error: PCA results file not found in $DIRECTORY."
        exit 1
    fi
done

echo "All PCA plots created successfully."


In [ ]:
## Lancer le script

sbash pca_plot.sh

Utilisons des outils afin d'interprêter les résultats de la PCA ou les plots obtenus

    Pour celà, nous disposons de trois outils fréquemment utilisés

Résumé

    Utilisez k-means pour partitionner les données en supposant un nombre de clusters.
    Appliquez DBSCAN pour détecter les clusters denses et identifier des outliers.
    Utilisez la méthode du coude et l’indice de silhouette pour déterminer le nombre optimal de clusters.

In [ ]:
## Se déplacer dans le répertoire créé

cd /scratch/MOryzae/SCRIPTS

Sauvegarder le script python dans le répertoire SCRIPTS avant le script sbatch

Le script sbatch va alors utiliser le contenu du script python pour l'exécution

In [ ]:
## Ouvrir l'éditeur de texte nano

nano clustering_analysis.py

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# Check for input arguments
if len(sys.argv) != 3:
    print("Usage: python clustering_analysis.py <PCA_FILE> <OUTPUT_DIR>")
    sys.exit(1)

# Input and output paths
pca_file = sys.argv[1]
output_dir = sys.argv[2]

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load PCA data
try:
    print(f"Loading PCA data from {pca_file}...")
    data = pd.read_csv(pca_file, sep=r'\s+', header=None)
    data.columns = ['FID', 'IID', 'PC1', 'PC2', 'PC3']
    X = data[['PC1', 'PC2', 'PC3']]
except Exception as e:
    print(f"Error loading PCA data: {e}")
    sys.exit(1)

# Determine optimal number of clusters using the elbow method
print("Calculating optimal number of clusters (Elbow Method)...")
inertia = []
k_values = range(1, 10)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
elbow_plot_path = os.path.join(output_dir, 'elbow_method.png')
plt.savefig(elbow_plot_path)
print(f"Elbow Method plot saved to {elbow_plot_path}")

# Apply k-means clustering with k=3 (or adjust based on elbow results)
print("Applying k-means clustering...")
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
data['Cluster'] = kmeans.fit_predict(X)

# Sauvegarder les résultats avec les clusters dans un fichier CSV
output_csv = os.path.join(output_dir, 'clustered_data.csv')
data.to_csv(output_csv, index=False)
print(f"Données annotées avec clusters sauvegardées dans {output_csv}")


# Visualize k-means clusters in 3D
print("Generating 3D k-means plot...")
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(data['PC1'], data['PC2'], data['PC3'], c=data['Cluster'], cmap='viridis', s=100)
plt.colorbar(scatter, ax=ax)
ax.set_title('k-means Clustering (k=3)')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
kmeans_plot_path = os.path.join(output_dir, 'kmeans_pca_plot_3d.png')
plt.savefig(kmeans_plot_path)
print(f"3D k-means plot saved to {kmeans_plot_path}")

# Apply DBSCAN clustering
print("Applying DBSCAN clustering...")
dbscan = DBSCAN(eps=0.1, min_samples=3)
data['DBSCAN_Cluster'] = dbscan.fit_predict(X)

# Visualize DBSCAN clusters in 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(data['PC1'], data['PC2'], data['PC3'], c=data['DBSCAN_Cluster'], cmap='plasma', s=100)
plt.colorbar(scatter, ax=ax)
ax.set_title('DBSCAN Clustering')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
dbscan_plot_path = os.path.join(output_dir, 'dbscan_pca_plot_3d.png')
plt.savefig(dbscan_plot_path)
print(f"3D DBSCAN plot saved to {dbscan_plot_path}")

# Calculate silhouette scores for different k
print("Calculating silhouette scores...")
silhouette_scores = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, labels))

plt.figure(figsize=(8, 5))
plt.plot(range(2, 10), silhouette_scores, marker='o')
plt.title('Silhouette Scores')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
silhouette_plot_path = os.path.join(output_dir, 'silhouette_scores.png')
plt.savefig(silhouette_plot_path)
print(f"Silhouette Scores plot saved to {silhouette_plot_path}")

print("Clustering analysis completed.")


In [ ]:
## Ouvrir l'éditeur de texte nano

nano pca_learning.sh

In [ ]:
#!/bin/bash

############# SLURM Configuration ##############
#SBATCH --job-name=pca_learning
#SBATCH -p normal
#SBATCH -c 8
#SBATCH --nodelist=node20

#################################################

# Load necessary modules
module load python/3.12.0

# Define directories
PCA_RESULTS_DIR="/scratch/MOryzae/PLINK"
OUTPUT_PLOT_DIR="/scratch/MOryzae/PLINK/PCA"

# Create output directory if it doesn't exist
mkdir -p "$OUTPUT_PLOT_DIR"

# List of subdirectories to process
DIRECTORIES=("plink")

# Loop over each directory
for DIRECTORY in "${DIRECTORIES[@]}"; do
    PCA_FILE="$PCA_RESULTS_DIR/$DIRECTORY/dataset.eigenvec"
    OUTPUT_DIR="$OUTPUT_PLOT_DIR"

    # Check if the PCA file exists
    if [[ -f "$PCA_FILE" ]]; then
        mkdir -p "$OUTPUT_DIR"
        echo "Processing PCA file: $PCA_FILE"

        # Run the Python script for clustering and plotting
        python3 clustering_analysis.py "$PCA_FILE" "$OUTPUT_DIR"

        echo "Processing completed for directory: $DIRECTORY"
    else
        echo "PCA file not found: $PCA_FILE"
    fi
done


In [ ]:
## Lancer le script

sbash pca_learning.sh

In [ ]:
#### Copier les données vers le NAS

scp -r /scratch/MOryzae/PLINK/ san:/projects/medium/CIBiG_MOryzae/

#### Analyser les plots produits et interprêter

## __DAPC__

Lancer ces scripts R sur votre machine locale

In [ ]:
## Ouvrir l'éditeur de texte nano

nano dapc_analysis.R

In [ ]:
# Charger les bibliothèques nécessaires
if (!requireNamespace("adegenet")) install.packages("adegenet")
if (!requireNamespace("factoextra")) install.packages("factoextra") # Pour le clustering
if (!requireNamespace("ggplot2")) install.packages("ggplot2")
if (!requireNamespace("dplyr")) install.packages("dplyr")

library(adegenet)
library(factoextra)
library(ggplot2)
library(dplyr)

# Définir les chemins d'entrée et de sortie
input_file <- "/home/name/Documents/Projet_CIBiG/Mentoring_Project/Results/PLINK/plink/dataset.eigenvec"
output_dir <- "/home/name/Documents/Projet_CIBiG/Mentoring_Project/Results/PLINK/DAPC"

# Créer le répertoire de sortie si nécessaire
if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE)
  cat("Répertoire de sortie créé :", output_dir, "\n")
}

# Charger les données PCA
cat("Chargement des données PCA...\n")
pca_data <- read.table(input_file, header = FALSE)
colnames(pca_data) <- c("FID", "IID", "PC1", "PC2", "PC3")  # Modifier selon vos colonnes
X <- pca_data %>% select(PC1, PC2, PC3)

# Étape 1 : Clustering des individus (k-means)
cat("Application de k-means pour générer des groupes...\n")
set.seed(42)  # Pour assurer la reproductibilité
n_clusters <- 3  # Ajustez selon vos besoins ou utilisez la méthode du coude (voir ci-dessous)
kmeans_result <- kmeans(X, centers = n_clusters)

# Ajouter les groupes au jeu de données
pca_data$Group <- as.factor(kmeans_result$cluster)

# Sauvegarder les groupes dans un fichier CSV
groups_file <- file.path(output_dir, "groups_kmeans.csv")
write.csv(pca_data, groups_file, row.names = FALSE)
cat("Groupes sauvegardés dans :", groups_file, "\n")

# Étape optionnelle : Déterminer le nombre optimal de clusters
# Méthode du coude
cat("Déterminer le nombre optimal de clusters avec la méthode du coude...\n")
fviz_nbclust(X, kmeans, method = "wss") +
  labs(title = "Méthode du coude pour déterminer k")

# Étape 2 : DAPC
cat("Optimisation du nombre de PCs pour DAPC...\n")
dapc_initial <- dapc(X, pca_data$Group)
optimal_pcs <- optim.a.score(dapc_initial)
n_pcs <- optimal_pcs$n.pca
cat(paste("Nombre optimal de PCs :", n_pcs, "\n"))

# Réaliser la DAPC avec le nombre optimal de PCs
dapc_result <- dapc(X, pca_data$Group, n.pca = n_pcs)

# Étape 3 : Visualisation des résultats
cat("Génération du graphique des clusters DAPC...\n")
scatter_file <- file.path(output_dir, "dapc_scatter.png")
png(scatter_file, width = 800, height = 600)
scatter(dapc_result, scree.da = TRUE, posi.da = "bottomleft", scree.pca = TRUE)
dev.off()
cat("Graphique DAPC sauvegardé dans :", scatter_file, "\n")

# Graphique ggplot des clusters
dapc_df <- data.frame(dapc_result$ind.coord) %>%
  mutate(Group = pca_data$Group)  # Ajouter les groupes au dataframe

ggplot_file <- file.path(output_dir, "dapc_ggplot.png")
gg <- ggplot(dapc_df, aes(x = LD1, y = LD2, color = Group)) +
  geom_point(size = 3, alpha = 0.8) +
  theme_minimal() +
  labs(title = "Clusters DAPC", x = "Discriminant Axis 1", y = "Discriminant Axis 2")
ggsave(ggplot_file, plot = gg, width = 8, height = 6)
cat("Graphique ggplot DAPC sauvegardé dans :", ggplot_file, "\n")

# Étape 4 : Contributions des variables
cat("Visualisation des contributions des variables...\n")
loading_file <- file.path(output_dir, "dapc_loadings.png")
png(loading_file, width = 800, height = 600)
loadingplot(dapc_result$var.contr, axis = 1, threshold = 0.005, lab.jitter = 1)
dev.off()
cat("Graphique des contributions sauvegardé dans :", loading_file, "\n")

# Étape 5 : Sauvegarder les résultats
results_file <- file.path(output_dir, "dapc_results_with_clusters.csv")
write.csv(dapc_df, results_file, row.names = FALSE)
cat("Résultats DAPC sauvegardés dans :", results_file, "\n")


In [ ]:
source("/path/to/working dorectory/on your laptop/dapc_analysis.R")